In [ ]:
#| default_exp featime

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from fastcore.test import *
from fastcore.utils import *
import pandas as pd

## Função mode

In [ ]:
#|  export
def mode(x):
    return pd.Series.mode(x).values[0]

A função mode pode ser utilizada na criação de variáveis categóricas, ela representa a moda dos valores. Note que se existir mais de uma moda, por padrão a função retorna o primeiro valor em ordem alfabética.

Exemplo de utilização da função moda

In [ ]:
lista1 = pd.DataFrame({'coluna1': ['a', 'b', 'b', 'b', 'c']})
mode(lista1['coluna1'])

'b'

Exemplo de moda com dois valores

In [ ]:
lista1 = pd.DataFrame({'coluna1': ['b', 'b', 'a', 'a', 'c']})
mode(lista1['coluna1'])

'a'

## Criação de variáveis

In [ ]:
#| export
def featime_in_time(df_, # Dataframe
                        id_cols, # Colunas de agrupamento como id e safra
                        trns_time_var:str, # Data correspondente a variável 
                        ref_time_var:str, # Data que será a referência para calcular as janelas de tempo 
                        value_vars:list, # Lista das variáveis que serão calculadas as operações
                        window:list, # Lista contendo a janela para calcular as variáveis 
                        operations:list # Lista das operações a serem calculadas
                        ): 
    
    df = df_.copy()
    df[trns_time_var] = pd.to_datetime(df[trns_time_var], format='%Y%m%d')
    df[ref_time_var] = pd.to_datetime(df[ref_time_var], format='%Y%m%d')
    df_spine =  df[id_cols].drop_duplicates()

    for var in value_vars:
        for size in window:
           
            time_cond = (df[trns_time_var] < df[ref_time_var]) & (df[trns_time_var] >= df[ref_time_var] + pd.DateOffset(months=size))
            df_grouped = df[time_cond].groupby(id_cols).agg({var:operations})
            df_grouped.columns = ['_'.join(col).strip()+'_'+str(abs(size))+'M' for col in df_grouped.columns.values]
            df_grouped.reset_index(inplace=True)
            df_spine = df_spine.merge(df_grouped,on=id_cols,how='left')

           
    return df_spine

Função utilizada para criar as variáveis categóricas e numéricas.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()